In [1]:
!pip install beautifulsoup4

### 1.2.2 Run BeautifulSoup
urlopen()을 임포트하고 html.read()를 호출하여 페이지의 HTML 콘텐츠를 얻었다.

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page1.html')
bs = BeautifulSoup(html.read(), 'html.parser')
print(bs.h1)

<h1>An Interesting Title</h1>


- 다음과 같이 .read()를 호출하지 않고 urlopen이 반환하는 파일 객체를 바로 사용할 수 도 있다.

In [6]:
bs = BeautifulSoup(html, 'html.parser')

## lxml
'지저분한' HTML 코드를 분석할 때 html.parser보다 좀 더 낫다.
lxml은 닫히지 않은 태그, 계층 구조가 잘못된 태그 등의 문제를 수정한다.

## html5lib
널리 쓰이는 구문 분석기이다.
lxml과 마찬가지로 잘못 쓰여진 HTML을 수정하고 구문 분석을 시도하는데, lxml보다 다양한 에러를 수정할 수 있다.

In [ ]:
bs = BeautifulSoup(html.read(), 'lxml')

In [8]:
bs = BeautifulSoup(html.read(), 'html5lib')

## 1.2.3 신뢰할 수 있는 연결과 예외 처리
웹은 엉망진창이다. 데이터 형식은 제대로 지켜지지 않고 웹사이트는 자주 다운되며 닫는 태그도 종종 빠져 있다.
이런 상황들을 예상하여 예외를 어떻게 처리할지 생각해보자.

In [9]:
html = urlopen('http://www.pythonscraping.com/pages/page1.html')

### 이 행에서 문제가 생길 수 있는 부분은 크게 두가지이다.
- 페이지를 찾을 수 없거나, URl 해석에 에러가 생긴 경우
- 서버를 찾을 수 없는 경우

In [13]:
from urllib.request import urlopen, HTTPError

try:
    html = urlopen('http://www.pythonscraping.com/pages/error.html')
except HTTPError as e:
    print(e) # null을 반환하거나, break 문을 실행하거나, 기타 다른 방법을 사용

HTTP Error 404: Not Found


서버를 전혀 찾을 수 없거나 페이지가 다운됐을 때, URL에 오타가 있을 떄 urlopen은 URLError 예외를 발생힌다.

In [17]:
from urllib.request import urlopen, HTTPError, URLError

try:
    html = urlopen('https://pythonscrapingthisurlopendoesnotexist.com')
except HTTPError as e:
    print(e)
except URLError as e:
    print('The Server could not be found!')
else:
    print("It Worked!")

The Server could not be found!


- 객체에 들어있는 태그에 접근할 때마다 그 태그가 실제 존재하는지 체크하는 것이 좋다.
- 존재하지 않는 태그에 접근을 시도하면 BeautifulSoup는 None 객체를 반환한다.
- 문제는 None 객체 자체에 태그가 있다고 가정하고 그 태그에 접근하려면 AttributeError가 발생한다.

In [18]:
print(bs.nonExistentTag)

None


/opt/anaconda3/lib/python3.8/site-packages/bs4/element.py:1444: UserWarning: .nonExistentTag is deprecated, use .find("nonExistent") instead. If you really were looking for a tag called nonExistentTag, use .find("nonExistentTag")
  warnings.warn(


In [19]:
print(bs.nonExistentTag.someTag)

/opt/anaconda3/lib/python3.8/site-packages/bs4/element.py:1444: UserWarning: .nonExistentTag is deprecated, use .find("nonExistent") instead. If you really were looking for a tag called nonExistentTag, use .find("nonExistentTag")
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'someTag'

In [20]:
# 이를 해결하기 위해 두 상황을 명시적으로 체크함
try:
    badContent = bs.nonExistingTag.anotherTag
except AttributeError as e:
    print("Tag was not found")
else:
    if badContent == None:
        print("Tag was not found")
    else:
        print(badContent)

Tag was not found


/opt/anaconda3/lib/python3.8/site-packages/bs4/element.py:1444: UserWarning: .nonExistingTag is deprecated, use .find("nonExisting") instead. If you really were looking for a tag called nonExistingTag, use .find("nonExistingTag")
  warnings.warn(


In [21]:
from urllib.request import urlopen, HTTPError
from bs4 import BeautifulSoup

def getTitle(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None
    try:
        bs = BeautifulSoup(html.read(), 'html.parser')
        title = bs.body.h1
    except AttributeError as e:
        return None
    return title

title = getTitle('http://www.pythoscraping.com/pages/page1.html')
if title == None:
    print("Title could not be found")
else:
    print(title)

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>